In [3]:
import numpy as np
import matplotlib
%matplotlib inline

import h5py

/Users/Jin/anaconda2/envs/pytorch/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader

## Import Model

In [5]:
from FBPConvNet import FBPConvNet

## Import train_net

In [6]:
# Import train_net
import sys
sys.path.append('../')
from net_utils import train_net

# Load Data

In [7]:
# set path to data
pathtodata = '../data/LineEllipses/LineEllipses20.hdf5'
batch_size = 4

In [8]:
f = h5py.File(pathtodata,'r')
trainingset = f['training']
training_data = f['training/training_data']
training_labels = f['training/training_labels']

In [9]:
trainset = TensorDataset(torch.Tensor(training_data[:5000]).unsqueeze(1),
                        torch.Tensor(training_labels[:5000]).unsqueeze(1))
testset = TensorDataset(torch.Tensor(training_data[5000:8000]).unsqueeze(1),
                        torch.Tensor(training_labels[5000:8000]).unsqueeze(1))

In [10]:
f.close()

In [11]:
trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(testset, batch_size=4,shuffle=False)

# Train Net

In [12]:
num_epochs = 500
GPU = torch.cuda.is_available()
print('Using GPU:',GPU)

# save path for trained models
weightpath = './weights/'

# save every x epochs
save_epoch = 50

Using GPU: False


In [19]:
net = FBPConvNet()
if GPU:
    net = net.cuda()

In [ ]:
train_net(net,trainloader,num_epochs,GPU,weightpath=weightpath,
         save_epoch=save_epoch,lr=0.01,momentum=0.99,saveweights=True)

# Test Net

In [20]:
weightspath = '../epoch_500.weights'
checkpoint = torch.load(weightspath)
net.load_state_dict(checkpoint)

In [22]:
testiter = iter(testloader)

In [ ]:
imgs,labels = testiter.next()
num = imgs.size(0)
est_imgs = net.forward(Variable(imgs))
plt.figure(figsize=(12,4*num))
for i in range(num):
    plt.subplot(num,3,3*i+1)
    plt.imshow(labels[i,0,:,:].numpy())
    plt.title('Ground Truth %i' % (i))
    break